**Design and implement a CNN for Image Classification a) Select a suitable image
classification dataset (medical imaging, agricultural, etc.). b) Optimized with different
hyper-parameters including learning rate, filter size, no. of layers, optimizers, dropouts,
e**tc.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load and preprocess the dataset
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'beans',
    split=['train', 'validation', 'test'],
    as_supervised=True,  # returns (image, label) pairs
    with_info=True       # returns dataset metadata
)

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, [128, 128]) / 255.0  # Resize and normalize
    return image, label

BATCH_SIZE = 32
ds_train = ds_train.map(preprocess).batch(BATCH_SIZE)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE)

def create_model(learning_rate=0.001, filter_size=(3, 3), num_layers=2, dropout_rate=0.5, optimizer='adam'):
    model = tf.keras.Sequential()

    # First Conv2D layer
    model.add(tf.keras.layers.Conv2D(32, filter_size, activation='relu', input_shape=(128, 128, 3)))
    model.add(tf.keras.layers.MaxPooling2D())

    # Add additional Conv2D layers based on num_layers parameter
    for _ in range(num_layers - 1):
        model.add(tf.keras.layers.Conv2D(64, filter_size, activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D())

    # Flatten and add Dense layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    model.compile(optimizer=optimizer(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


# Define a list of hyperparameter combinations
hyperparameters = [
    {'learning_rate': 0.001, 'filter_size': (3, 3), 'num_layers': 2, 'dropout_rate': 0.5, 'optimizer': tf.keras.optimizers.Adam},
    {'learning_rate': 0.002, 'filter_size': (5, 5), 'num_layers': 3, 'dropout_rate': 0.3, 'optimizer': tf.keras.optimizers.Adam},
    {'learning_rate': 0.003, 'filter_size': (5, 5), 'num_layers': 2, 'dropout_rate': 0.5, 'optimizer': tf.keras.optimizers.SGD},
]

# Run the models with different hyperparameters
for params in hyperparameters:
    print(f"\nTraining with hyperparameters: {params}")

    model = create_model(
        learning_rate=params['learning_rate'],
        filter_size=params['filter_size'],
        num_layers=params['num_layers'],
        dropout_rate=params['dropout_rate'],
        optimizer=params['optimizer']
    )

    # Train the model
    model.fit(ds_train, epochs=5, validation_data=ds_val)

    # Evaluate on test data
    test_loss, test_acc = model.evaluate(ds_test)
    print(f"Test Accuracy: {test_acc:.2f}")
